<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Explorando_dados_do_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="20%" src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/_imagens/Titanic4.jpg?raw=true" align="center">

In [0]:
# teste

![alt text](https://drive.google.com/uc/d/1tR4M2maLGBTun3N4yyq-dTzDdQPucFna/view?usp=sharing)